In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from speckit.core import SpectrumAnalyzer
from speckit.noise import fftnoise, band_limited_noise
import multiprocessing

In [ ]:
N  = int(1e6)
fs = 2

x1 = 1e-6*band_limited_noise(1e-6, 1e0, N, fs)\
    + 1e3*band_limited_noise(1e-3, 10e-3, N, fs)\
        + 1e3*band_limited_noise(50e-3, 60e-3, N, fs)\

fig, ax = plt.subplots(figsize=(16,4), dpi=150)
ax.set_xlabel('Sample')
ax.set_ylabel('Signal')
ax.plot(x1, color='black')
plt.show()

f1, psd = signal.welch(x1, fs=fs, window=("kaiser", 30), nperseg=int(N), noverlap=int(N/2), scaling='density', return_onesided=True)

fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Spectral density (units $/\sqrt{\rm Hz}$)')
ax.loglog(f1, psd, color='black')
fig.tight_layout()
plt.show()

In [ ]:
# 1. Configure the analyzer
print("Configuring SpectrumAnalyzer...")
analyzer = SpectrumAnalyzer(
    data=x1,
    fs=fs,
    bmin=1.0,
    order=-1,
    win="Kaiser",
    Jdes=1000,
    psll=200,
    verbose=True
)

# 2. (Optional) Inspect the plan before computation
plan = analyzer.plan()
print(f"\nGenerated a plan with {plan['nf']} frequency bins.")

# 3. Create a multiprocessing pool and run the computation
pool = multiprocessing.Pool()
print("\nStarting computation...")
result_advanced = analyzer.compute(pool=pool)
pool.close()
pool.join()

print("\nComputation complete!")
print(f"Result object type: {type(result_advanced)}")

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Spectral density (units $/\sqrt{\rm Hz}$)')
ax.loglog(f1, np.sqrt(psd), label="Welch", color='gray')
result_advanced.plot(which='asd', ax=ax, label='LPSD')
ax.legend()
fig.tight_layout()
plt.show()